# 1. Import Library

In [1]:
import dask.array as da
import dask.dataframe as dd
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import average_precision_score
import tensorflow as tf
from tensorflow.keras import layers as tkl
from tensorflow_addons.callbacks import TimeStopping

/home/aravindh/miniconda3/envs/fog_venv/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# 2. Setting Precision

In [2]:
tf.keras.backend.set_floatx('float32')
tf.keras.backend.floatx()

'float32'

# 4. Data Transformations

## a.) Data Selections Transformation

In [3]:
def get_stats(data):
    temp = data
    data['AccV_AccML'] = temp.AccV + temp.AccML
    data['row_mean'] = temp.mean(axis=1)
    data['Stride'] = temp.AccV + temp.AccML + temp.AccAP
    data['row_median'] = temp.map_partitions(lambda df: df.median(axis=1))
    return data

In [4]:
def data_select(data):
    x = data.iloc[:, 1:-3]
    y = data.iloc[:, -3:]
    x.AccV = x.AccV - x.AccV.shift(1)
    x.AccML = x.AccML - x.AccML.shift(1)
    x.AccAP = x.AccAP - x.AccAP.shift(1)
    x = get_stats(x)
    x = x.fillna(0)
    return x ,y

## b.) Data Structuring Transformation

In [5]:
def tensor_restructure(X, Y, win_size=32):
    data_img_list = []
    data_res_list = []
    for part in range(X.npartitions):
        df = X.partitions[part].compute()
        res_df = Y.partitions[part].compute()

        for end in range(win_size, res_df.shape[0]):
            sub_res_df = res_df.iloc[end - win_size: end, :]
            sub_df = df.iloc[end - win_size: end, :]
            data_img_list.append(sub_df.values)
            data_res_list.append(sub_res_df.values[1::2])
            
    data_res_list = np.array(data_res_list)
    data_img_list = np.array(data_img_list)
    return data_img_list, data_res_list

# 5. tdcsfog Data Selection

## a.) Setting Data Path

In [6]:
main_path = '../data/raw'

## b.) Loading tdcsfog metadata

In [7]:
tdcsfog =  pd.read_csv(main_path + '/tdcsfog_metadata.csv')
meta = tdcsfog
meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833 entries, 0 to 832
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Id          833 non-null    object
 1   Subject     833 non-null    object
 2   Visit       833 non-null    int64 
 3   Test        833 non-null    int64 
 4   Medication  833 non-null    object
dtypes: int64(2), object(3)
memory usage: 32.7+ KB


## c.) Loading subject Data

In [8]:
subject = pd.read_csv(main_path + '/subjects.csv')
subject.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Subject       173 non-null    object 
 1   Visit         111 non-null    float64
 2   Age           173 non-null    int64  
 3   Sex           173 non-null    object 
 4   YearsSinceDx  173 non-null    float64
 5   UPDRSIII_On   172 non-null    float64
 6   UPDRSIII_Off  132 non-null    float64
 7   NFOGQ         173 non-null    int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 10.9+ KB


In [9]:
subject = subject.fillna(0)
subject.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Subject       173 non-null    object 
 1   Visit         173 non-null    float64
 2   Age           173 non-null    int64  
 3   Sex           173 non-null    object 
 4   YearsSinceDx  173 non-null    float64
 5   UPDRSIII_On   173 non-null    float64
 6   UPDRSIII_Off  173 non-null    float64
 7   NFOGQ         173 non-null    int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 10.9+ KB


## d.) Loading event Data

In [10]:
events = pd.read_csv(main_path + '/events.csv')
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3544 entries, 0 to 3543
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Id          3544 non-null   object 
 1   Init        3544 non-null   float64
 2   Completion  3544 non-null   float64
 3   Type        2502 non-null   object 
 4   Kinetic     2502 non-null   float64
dtypes: float64(3), object(2)
memory usage: 138.6+ KB


In [11]:
events["Event_Duration"] = events["Completion"] - events["Init"]
events = events.dropna()
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2502 entries, 0 to 3543
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              2502 non-null   object 
 1   Init            2502 non-null   float64
 2   Completion      2502 non-null   float64
 3   Type            2502 non-null   object 
 4   Kinetic         2502 non-null   float64
 5   Event_Duration  2502 non-null   float64
dtypes: float64(4), object(2)
memory usage: 136.8+ KB


## e.) Data Merging

In [12]:
meta = pd.merge(meta, subject, how='inner', on='Subject')
meta.head()

,Id,Subject,Visit_x,Test,Medication,Visit_y,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ
0,003f117e14,4dc2f8,3,2,on,0.0,68,F,9.0,17.0,15.0,15
1,15671a03ca,4dc2f8,2,2,off,0.0,68,F,9.0,17.0,15.0,15
2,2ca9bd3533,4dc2f8,2,3,on,0.0,68,F,9.0,17.0,15.0,15
3,3c535f4851,4dc2f8,4,1,on,0.0,68,F,9.0,17.0,15.0,15
4,5a321dde31,4dc2f8,2,3,off,0.0,68,F,9.0,17.0,15.0,15


In [13]:
meta = pd.merge(meta, events, how='inner', on='Id')
meta = meta.sort_values(['Id', 'Init'], ignore_index=True)
meta.head()

,Id,Subject,Visit_x,Test,Medication,Visit_y,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ,Init,Completion,Type,Kinetic,Event_Duration
0,003f117e14,4dc2f8,3,2,on,0.0,68,F,9.0,17.0,15.0,15,8.61312,14.7731,Turn,1.0,6.15998
1,009ee11563,f62eec,4,2,on,0.0,71,M,10.0,42.0,0.0,24,11.38470,41.1847,Turn,1.0,29.80000
2,009ee11563,f62eec,4,2,on,0.0,71,M,10.0,42.0,0.0,24,54.66470,58.7847,Turn,1.0,4.12000
3,011322847a,231c3b,2,2,on,0.0,67,M,12.0,27.0,28.0,19,28.09660,30.2966,Turn,1.0,2.20000
4,01d0fe7266,231c3b,2,1,off,0.0,67,M,12.0,27.0,28.0,19,30.31840,31.8784,Turn,1.0,1.56000


## f.) Data Stats

In [14]:
# sns.pairplot(data=meta, hue='Type', height=3)
# plt.show()
# meta.head()

## g.) Data ID Selection

### (i) Training IDs

In [15]:
sh_ids = meta.loc[meta.Type == 'StartHesitation', "Id"].unique()
walk_ids = meta.loc[meta.Type == 'Walking', "Id"].unique()
sh_ids = [ids for ids in sh_ids if ids in walk_ids]
print(sh_ids)

['10e83d6550', '150e8227fc', '19fa6cec5a', '2365c918c9', '35bd42e628', '3ba3590a08', '45a205e6ce', '46300e025f', '6670936848', '692e9e05ef', '6d392739be', '6d9b1fc826', '714dc454eb', '76c7edf878', '7fb91164ad', '81262644e7', '8126370646', '8969714d63', '9b378f315c', '9d6017c17e', '9e6056e271', 'acf784d6d7', 'b670d1cddd', 'babcb58d97', 'be6e0ddcef', 'c36c3860e7', 'c784d2f5d6', 'c8622e0019', 'cd7bca4654', 'd98358a75f', 'e318182fe2', 'eaa4651a87', 'f60bf42bd8', 'ff92d9244d']


In [16]:
start = meta.loc[meta.Id.isin(sh_ids)]
start = start.loc[start.Type != 'Turn']
start["Gap"] = start.Init.shift(-1) - start.Completion
start["Gap_Type"]  =  start.Type.shift(-1) + " - " + start.Type
start = start.loc[(start.Gap < 10) & (start.Gap > 0) & (start.Type == 'StartHesitation') & (start.Gap_Type == "Walking - StartHesitation")]
start = start.reset_index(drop=True)
start.head()

,Id,Subject,Visit_x,Test,Medication,Visit_y,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ,Init,Completion,Type,Kinetic,Event_Duration,Gap,Gap_Type
0,6670936848,79011a,3,2,on,0.0,73,M,7.0,37.0,46.0,20,3.52688,4.72688,StartHesitation,1.0,1.20,1.58,Walking - StartHesitation
1,692e9e05ef,4f13b4,2,1,off,0.0,68,F,12.0,22.0,39.0,22,3.60094,6.36094,StartHesitation,1.0,2.76,2.72,Walking - StartHesitation
2,6d392739be,79011a,2,3,on,0.0,73,M,7.0,37.0,46.0,20,2.40000,6.16000,StartHesitation,1.0,3.76,1.24,Walking - StartHesitation
3,81262644e7,b19f77,19,1,on,0.0,82,F,15.0,54.0,68.0,21,44.15280,47.85280,StartHesitation,0.0,3.70,5.72,Walking - StartHesitation
4,9e6056e271,a80ae4,19,1,on,0.0,67,M,5.0,30.0,0.0,24,75.11340,80.87340,StartHesitation,0.0,5.76,2.40,Walking - StartHesitation


In [18]:
turn_ids = [ids for ids in meta.loc[meta.Type == 'Turn', "Id"].unique() if ids in walk_ids]
print(turn_ids)

['052a913d06', '07bd2e7fbc', '0b77238a70', '10e83d6550', '146bd622af', '150e8227fc', '19fa6cec5a', '2365c918c9', '2ca9bd3533', '2d481ad987', '311e2722d2', '3291d59f24', '35bd42e628', '3ba3590a08', '3cd4c680cb', '441362cade', '45a205e6ce', '46300e025f', '4ef4e8edef', '4f619d853b', '616371ed4d', '6670936848', '6698568138', '692e9e05ef', '6c39f4d317', '6d392739be', '714dc454eb', '7468af035f', '74de894ecf', '76c7edf878', '7fb91164ad', '81262644e7', '8126370646', '8969714d63', '9166ea61ac', '9b378f315c', '9d6017c17e', '9e0b9001cf', '9e6056e271', '9eed1eba0b', 'acf784d6d7', 'b4368821a4', 'b4a40c9efe', 'b670d1cddd', 'be6e0ddcef', 'c36c3860e7', 'c784d2f5d6', 'c8622e0019', 'ccf39d0b9e', 'cd7bca4654', 'd98358a75f', 'dcc5c991b5', 'e2b8e7b507', 'e318182fe2', 'eaa4651a87', 'eac8e044ca', 'ef4fd1bbb4', 'f60bf42bd8', 'f7993aac2f', 'ff92d9244d']


In [19]:
turn = meta.loc[meta.Id.isin(turn_ids)]
turn = turn.loc[turn.Type != 'StartHesitation']
turn["Gap"] = (turn.Init - turn.Completion.shift(1)).abs()
turn["Gap_Type"]  = turn.Type + " - "+ turn.Type.shift(1)
turn = turn.loc[(turn.Gap < 10) & (turn.Gap > 0) & (turn.Type == 'Turn') & (turn.Gap_Type.isin(["Turn - Walking", "Walking - Turn"]))]
turn["Event_Duration"] = turn["Event_Duration"].map(lambda x: abs(start["Event_Duration"].mean() - x))
turn = turn.sort_values("Event_Duration", ignore_index=True)
turn = turn.drop_duplicates("Id")
turn = turn.head(8)
turn.head()

,Id,Subject,Visit_x,Test,Medication,Visit_y,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ,Init,Completion,Type,Kinetic,Event_Duration,Gap,Gap_Type
0,9eed1eba0b,48fd62,13,3,on,0.0,81,M,1.0,42.0,30.0,16,115.4800,121.6400,Turn,1.0,0.235005,7.18,Turn - Walking
1,c784d2f5d6,2a39f8,2,1,off,0.0,67,M,6.0,16.0,61.0,21,51.8600,57.2200,Turn,1.0,0.564995,3.00,Turn - Walking
2,0b77238a70,2a39f8,3,1,off,0.0,67,M,6.0,16.0,61.0,21,45.1669,49.8469,Turn,1.0,1.244995,3.48,Turn - Walking
3,2d481ad987,79011a,2,1,off,0.0,73,M,7.0,37.0,46.0,20,79.4228,86.7428,Turn,1.0,1.395005,3.48,Turn - Walking
4,3291d59f24,a03db7,5,2,off,0.0,56,M,11.0,27.0,55.0,19,39.0491,42.9291,Turn,1.0,2.044995,4.04,Turn - Walking


In [20]:
processed_ids = meta.loc[meta.Id.isin(start.Id) | meta.Id.isin(turn.Id)]
processed_ids = processed_ids.reset_index(drop=True)
processed_ids.head()

,Id,Subject,Visit_x,Test,Medication,Visit_y,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ,Init,Completion,Type,Kinetic,Event_Duration
0,0b77238a70,2a39f8,3,1,off,0.0,67,M,6.0,16.0,61.0,21,8.40687,10.8069,Walking,1.0,2.40003
1,0b77238a70,2a39f8,3,1,off,0.0,67,M,6.0,16.0,61.0,21,11.92690,30.5269,Turn,1.0,18.60000
2,0b77238a70,2a39f8,3,1,off,0.0,67,M,6.0,16.0,61.0,21,39.04690,41.6869,Walking,1.0,2.64000
3,0b77238a70,2a39f8,3,1,off,0.0,67,M,6.0,16.0,61.0,21,45.16690,49.8469,Turn,1.0,4.68000
4,0b77238a70,2a39f8,3,1,off,0.0,67,M,6.0,16.0,61.0,21,50.84690,54.2069,Walking,1.0,3.36000


In [21]:
data_ids = list(map(lambda x: main_path + '/train/tdcsfog/' + x + '.csv', processed_ids.Id.unique()))
selected_data = dd.read_csv(data_ids)
selected_data.head()

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
0,0,-9.552306,0.467110,-0.661416,0,0,0
1,1,-9.550023,0.464809,-0.647864,0,0,0
2,2,-9.551571,0.458210,-0.622576,0,0,0
3,3,-9.544351,0.455957,-0.635837,0,0,0
4,4,-9.550601,0.462419,-0.646041,0,0,0


In [22]:
x_train, y_train = data_select(selected_data)
y_train.head()

,StartHesitation,Turn,Walking
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [23]:
x_train.head()

,AccV,AccML,AccAP,AccV_AccML,row_mean,Stride,row_median
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.002283,-0.002300,0.013551,-0.000017,0.003379,0.013534,0.002831
2,-0.001549,-0.006599,0.025289,-0.008147,0.002248,0.017141,0.000350
3,0.007220,-0.002253,-0.013261,0.004967,-0.000832,-0.008294,-0.001542
4,-0.006249,0.006461,-0.010204,0.000212,-0.002445,-0.009993,-0.004347


### (ii) Testing Validation IDs

In [24]:
test_val_ids = [ids for ids in meta.Id.unique() if (ids not in start.Id) and (ids not in turn.Id) and (ids in sh_ids) and (ids in turn_ids) and (ids in walk_ids)]
test_val_ids.sort()
print(test_val_ids)

['10e83d6550', '150e8227fc', '19fa6cec5a', '2365c918c9', '35bd42e628', '3ba3590a08', '45a205e6ce', '46300e025f', '6670936848', '692e9e05ef', '6d392739be', '714dc454eb', '76c7edf878', '7fb91164ad', '81262644e7', '8126370646', '8969714d63', '9b378f315c', '9d6017c17e', '9e6056e271', 'acf784d6d7', 'b670d1cddd', 'be6e0ddcef', 'c36c3860e7', 'c784d2f5d6', 'c8622e0019', 'cd7bca4654', 'd98358a75f', 'e318182fe2', 'eaa4651a87', 'f60bf42bd8', 'ff92d9244d']


In [25]:
test_val_ids_path = list(map(lambda x: main_path + '/train/tdcsfog/' + x + '.csv', test_val_ids))
test_val_data = dd.read_csv(test_val_ids_path)
test_val_data.head()

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
0,0,-9.687117,-1.238445,-1.932528,0,0,0
1,1,-9.686484,-1.233967,-1.932522,0,0,0
2,2,-9.683087,-1.236049,-1.936234,0,0,0
3,3,-9.684048,-1.240766,-1.937733,0,0,0
4,4,-9.683254,-1.239990,-1.942938,0,0,0


In [26]:
test_data = test_val_data.partitions[:]
test_data.head()

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking
0,0,-9.687117,-1.238445,-1.932528,0,0,0
1,1,-9.686484,-1.233967,-1.932522,0,0,0
2,2,-9.683087,-1.236049,-1.936234,0,0,0
3,3,-9.684048,-1.240766,-1.937733,0,0,0
4,4,-9.683254,-1.239990,-1.942938,0,0,0


In [27]:
x_test, y_test = data_select(test_data)
y_test.head()

,StartHesitation,Turn,Walking
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [28]:
x_test.head()

,AccV,AccML,AccAP,AccV_AccML,row_mean,Stride,row_median
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000632,0.004479,0.000006,0.005111,0.002557,0.005117,0.003518
2,0.003397,-0.002082,-0.003712,0.001315,-0.000271,-0.002397,-0.001176
3,-0.000961,-0.004717,-0.001499,-0.005678,-0.003214,-0.007177,-0.003965
4,0.000794,0.000776,-0.005205,0.001570,-0.000516,-0.003635,0.000130


# 6. Modeling

## a.) Setting BATCH_SIZE & EPOCHS

In [29]:
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
# tpu_strategy = tf.distribute.TPUStrategy(tpu)
# batch_size = 16 * tpu_strategy.num_replicas_in_sync
gpu_count = len(tf.config.list_logical_devices('GPU'))
batch_size = 16*gpu_count
# batch_size = 32
epoch = 100
window = 32
input_size = (window, x_train.shape[1])
print(f'BATCH_SIZE: {batch_size}')
print(f'EPOCH: {epoch}')
print(f'WINDOW: {window}')
print(f'INPUT_SIZE: {input_size}')

BATCH_SIZE: 16
EPOCH: 100
WINDOW: 32
INPUT_SIZE: (32, 7)


## b.) Build

In [30]:
def convo_layers(ly_in):
    cl_in = tkl.Conv1D(filters=128, kernel_size=3, strides=1, activation='relu', name='Convo1D-in')(ly_in)
    ap1 = tkl.AveragePooling1D(pool_size=3, strides=1, name='AvgPooling1D-1')(cl_in)
    cl2 = tkl.Conv1D(filters=64, kernel_size=2, strides=1, activation='relu', name='Convo1D-2')(ap1)
    ap2 = tkl.AveragePooling1D(pool_size=2, strides=1, name='AvgPooling1D-2')(cl2)
    cl3 = tkl.Conv1D(filters=32, kernel_size=2, strides=1, activation='relu', name='Convo1D-3')(ap2)
    ap3 = tkl.AveragePooling1D(pool_size=2, strides=1, name='AvgPooling1D-3')(cl3)
    cl4 = tkl.Conv1D(filters=16, kernel_size=3, strides=1, activation='relu', name='Convo1D-4')(ap3)
    ap4 = tkl.AveragePooling1D(pool_size=3, strides=1, name='AvgPooling1D-4')(cl4)
    cl5 = tkl.Conv1D(filters=8, kernel_size=3, strides=1, activation='relu', name='Convo1D-5')(ap4)
    ap_out = tkl.AveragePooling1D(pool_size=3, strides=1, name='AvgPooling1D-out')(cl5)
    return ap_out

In [31]:
def dense_layers(ly_in):
    dl_in = tkl.Dense(units=256, activation='relu', name='Ann-in')(ly_in)
    dp_1 = tkl.Dropout(0.2, name='Drop-Ann-1')(dl_in)
    dl_2 = tkl.Dense(units=169, activation='relu', name='Ann-2')(dp_1)
    dp_2 = tkl.Dropout(0.2, name='Drop-Ann-2')(dl_2)
    dl_3 = tkl.Dense(units=121, activation='relu', name='Ann-3')(dp_2)
    dp_3 = tkl.Dropout(0.2, name='Drop-Ann-3')(dl_3)
    dl_4 = tkl.Dense(units=49, activation='relu', name='Ann-4')(dp_3)
    dp_4 = tkl.Dropout(0.2, name='Drop-Ann-4')(dl_4)
    dl_5 = tkl.Dense(units=25, activation='relu', name='Ann-5')(dp_4)
    dp_5 = tkl.Dropout(0.2, name='Drop-Ann-5')(dl_5)
    dl_6 = tkl.Dense(units=9, activation='relu', name='Ann-6')(dp_5)
    dp_out = tkl.Dropout(0.2, name='Drop-Ann-6')(dl_6)
    return dp_out
    

In [32]:
def build_tdcsfog_model(adapt_data, size):
    normalizer = tkl.Normalization()
    normalizer.adapt(adapt_data)
    in_ly = tf.keras.Input(shape=size)
    norm_out = normalizer(in_ly)
    bl_1 = convo_layers(norm_out)
    bl_2 = dense_layers(bl_1)
    out_ly = tkl.Dense(units=3, activation='softmax', name='Ann-out')(bl_2)
    ly_rs = tkl.Reshape((16, 3), name='Ann-out-shaper')(out_ly)
    return in_ly, ly_rs

## c.) Compilation

In [34]:
# with tpu_strategy.scope():

# mirrored_strategy = tf.distribute.MirroredStrategy()

# with mirrored_strategy.scope():
    
with tf.device("/GPU:0"):

    x_train, y_train = tensor_restructure(x_train, y_train)
    print(f'x_train: {x_train.shape}')
    print(f'y_train: {y_train.shape}')
    print('*************************')
    x_test, y_test = tensor_restructure(x_test, y_test)
    print(f'x_test: {x_test.shape}')
    print(f'y_test: {y_test.shape}')
    print('*************************')
    precision = tf.keras.metrics.Precision()
    model_in, model_out = build_tdcsfog_model(x_train, input_size)
    tdcsfog_model = tf.keras.Model(inputs=model_in, outputs=model_out)
    

tdcsfog_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', precision])
tdcsfog_model.summary()

x_train: (241899, 32, 7)
y_train: (241899, 16, 3)
*************************
x_test: (912475, 32, 7)
y_test: (912475, 16, 3)
*************************
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 7)]           0         
                                                                 
 normalization (Normalizatio  (None, 32, 7)            15        
 n)                                                              
                                                                 
 Convo1D-in (Conv1D)         (None, 30, 128)           2816      
                                                                 
 AvgPooling1D-1 (AveragePool  (None, 28, 128)          0         
 ing1D)                                                          
                                                                 
 Convo1D-2 (Conv1D)          (None, 27, 64)

## d.) Fitting

In [35]:
log_dir = '../logs/avg_pooling_6'
save_check_point = tf.keras.callbacks.ModelCheckpoint("../models/ModelCheckpoint/", monitor="val_precision", save_best_only=True, mode='max', save_weights_only=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True)
time_stopping = TimeStopping(seconds=60*60*4, verbose=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor="val_precision", patience=11, mode='max', restore_best_weights=True)
tdcsfog_model\
.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=epoch, callbacks=[save_check_point, tensorboard_callback, time_stopping, early_stopping_callback], verbose=1)

Epoch 1/100
15119/15119 [==============================] - 306s 20ms/step - loss: 0.5190 - accuracy: 0.5352 - precision: 0.5379 - val_loss: 0.7978 - val_accuracy: 0.5350 - val_precision: 0.5454
Epoch 2/100
15119/15119 [==============================] - 289s 19ms/step - loss: 0.4817 - accuracy: 0.5569 - precision: 0.5577 - val_loss: 0.8430 - val_accuracy: 0.5497 - val_precision: 0.5499
Epoch 3/100
15119/15119 [==============================] - 288s 19ms/step - loss: 0.4553 - accuracy: 0.5758 - precision: 0.5703 - val_loss: 0.8691 - val_accuracy: 0.5458 - val_precision: 0.5631
Epoch 4/100
15119/15119 [==============================] - 311s 21ms/step - loss: 0.4325 - accuracy: 0.5879 - precision: 0.5819 - val_loss: 0.9170 - val_accuracy: 0.5543 - val_precision: 0.5561
Epoch 5/100
15119/15119 [==============================] - 300s 20ms/step - loss: 0.4107 - accuracy: 0.5991 - precision: 0.5910 - val_loss: 0.9974 - val_accuracy: 0.5539 - val_precision: 0.5543
Epoch 6/100
 6338/15119 [=====

KeyboardInterrupt: 

## e.) Loading

In [34]:
tdcsfog_model.load_weights('../models/ModelCheckpoint/')

## f.) Metrics

In [35]:
metric_val = tdcsfog_model.evaluate(x_test, y_test, batch_size)
y_proba = tdcsfog_model.predict(x_test)
y_proba = tf.reshape(y_proba, (-1, 3))
y_test = tf.reshape(y_test, (-1, 3))

28515/28515 [==============================] - 169s 6ms/step


In [36]:
metric_val.append(np.mean(average_precision_score(y_test, y_proba, average=None)))
print(f'Loss: {metric_val[0]}')
print(f'Accuracy: {metric_val[1]}')
print(f'Precision: {metric_val[2]}')
print(f'Mean Average Precision(mAP): {metric_val[3]}')

Loss: 1.3885159492492676
Accuracy: 0.5555139183998108
Precision: 0.5564548969268799
Mean Average Precision(mAP): 0.3923160660511402


## g.) Saving

In [36]:
tdcsfog_model.save("../models/tdcsfog_model_avg_pooling_6")

INFO:tensorflow:Assets written to: ../models/tdcsfog_model_avg_pooling_6\assets


INFO:tensorflow:Assets written to: ../models/tdcsfog_model_avg_pooling_6\assets
